In [1]:
from pyflink.table import EnvironmentSettings, TableEnvironment, DataTypes, CsvTableSource
import datetime
from pyflink.table.expressions import col
from pyflink.table.window import Over, GroupWindow
from pyflink.table.expressions import col, UNBOUNDED_RANGE, CURRENT_RANGE
from pyflink.table.udf import udf
# create a batch TableEnvironment
env_settings = EnvironmentSettings.in_batch_mode()
table_env = TableEnvironment.create(env_settings)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/flink-1.15.0/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.7.7/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [3]:
# InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
# 536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 8:26,2.55,17850,United Kingdom
column_names = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 
                'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

column_types = [DataTypes.STRING(), DataTypes.STRING(), DataTypes.STRING(), DataTypes.DOUBLE(), 
                DataTypes.STRING(), DataTypes.DOUBLE(),DataTypes.STRING(), DataTypes.STRING()]

source = CsvTableSource(
   '../data/ecomm/1k-dataset.csv', #./data/3MINDIA.csv', #  './data',
    column_names,
    column_types,
    ignore_first_line=True,
    quote_character='"',
    lenient=True
)

table_env.register_table_source('invoices', source)

invoices = table_env.from_path('invoices')


##############################
print('\nRegistered Tables List')
print(table_env.list_tables())

print('\nFinancial Trxs Schema')
invoices.print_schema()



Registered Tables List
['invoices']

Financial Trxs Schema
(
  `InvoiceNo` STRING,
  `StockCode` STRING,
  `Description` STRING,
  `Quantity` DOUBLE,
  `InvoiceDate` STRING,
  `UnitPrice` DOUBLE,
  `CustomerID` STRING,
  `Country` STRING
)


In [5]:
invoices.fetch(3).execute().print()

+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                      InvoiceNo |                      StockCode |                    Description |                       Quantity |                    InvoiceDate |                      UnitPrice |                     CustomerID |                        Country |
+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                         536365 |                         85123A | WHITE HANGING HEART T-LIGHT... |                            6.0 |                12/01/2010 8:26 |                           2.55 |   

In [ ]:
#         .column_by_expression("rowtime", "CAST(f0 AS TIMESTAMP(3))") \



# datetime in string format for may 25 1999
input = '12/01/2010 8:26'
  
# format
format = '%d/%m/%Y %H:%M'
  
# convert from string format to datetime format
dt = datetime.datetime.strptime(input, format)
  
# get the date from the datetime using date() 
# function
print(dt)

new_format = '%Y-%m-%d %H:%M:%S' 
dt.strftime(new_format)

In [ ]:


def convertDateFormat2(input):
    # format
    format = '%d/%m/%Y %H:%M'
  
    # convert from string format to datetime format
    dt = datetime.datetime.strptime(input, format)
    new_format = '%Y-%m-%d %H:%M:%S' 
    return dt.strftime(new_format)


convertDateFormat = udf(convertDateFormat2, [DataTypes.STRING()], DataTypes.STRING())

 

In [ ]:

  
              
invoices2 = invoices.add_columns(convertDateFormat(col('InvoiceDate')).alias('CInvoiceDate'))\
                    .drop_columns(col('InvoiceDate'))\
                    .add_columns(col("CInvoiceDate").cast(DataTypes.TIMESTAMP(3)).alias("InvoiceDate"))\
                    .drop_columns(col('CInvoiceDate'))
                     

print('\n invoices2  Schema')
invoices2.print_schema()
invoices2.fetch(3).execute().print()